In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy.optimize import curve_fit
from scipy.stats import norm

#from mc_io_functions import load_mc_particles
#from mc_io_functions import load_mc_hits
#from mc_io_functions import load_mc_sensor_response

from invisible_cities.core.core_functions  import shift_to_bin_centers
from invisible_cities.core                 import fit_functions as fitf
#from invisible_cities.icaro.hst_functions import hist
#from invisible_cities.icaro.hst_functions import hist2d
#from invisible_cities.icaro.hst_functions import hist2d_profile
#from invisible_cities.icaro.hst_functions import labels


In [2]:
def hist(*args, **kwargs):
    """
    Create a figure and then the histogram
    """
    create_new_figure(kwargs)

    y, x, p = plt.hist(*args, **kwargs)
    return y, shift_to_bin_centers(x), p


def hist2d(*args, **kwargs):
    """
    Create a figure and then the histogram
    """
    create_new_figure(kwargs)

    z, x, y, p = plt.hist2d(*args, **kwargs)
    return z, shift_to_bin_centers(x), shift_to_bin_centers(y), p

def hist2d_profile(x, y, z, nbinx, nbiny, xrange, yrange, **kwargs):
    """
    Create a profile 2d of the data and plot it as an histogram.
    """
    x, y, z, ze = fitf.profileXY(x, y, z, nbinx, nbiny, xrange, yrange)
    plot_output = display_matrix(x, y, z, **kwargs)
    return ((x, y, z, ze), *plot_output)

def display_matrix(x, y, z, mask=None, **kwargs):
    """
    Display the matrix z using the coordinates x and y as the bin centers.
    """
    nx = x = np.size(x)
    ny = np.size(y)

    dx = (np.max(x) - np.min(x)) / nx
    dy = (np.max(y) - np.min(y)) / ny

    x_binning = np.linspace(np.min(x) - dx, np.max(x) + dx, nx + 1)
    y_binning = np.linspace(np.min(y) - dy, np.max(y) + dy, ny + 1)

    x_ = np.repeat(x, ny)
    y_ = np.tile  (y, nx)
    z_ = z.flatten()

    if mask is None:
        mask = np.ones_like(z_, dtype=bool)
    else:
        mask = mask.flatten()
    h  = hist2d(x_[mask], y_[mask], (x_binning,
                                     y_binning),
                weights = z_[mask],
                **kwargs)
    return h, plt.colorbar()

def create_new_figure(kwargs):
    if kwargs.setdefault("new_figure", True):
        plt.figure()
    del kwargs["new_figure"]

def labels(xlabel, ylabel, title=""):
    """
    Set x and y labels.
    """
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.title ( title)
    
def EnergyRes(x):
    mu = np.mean(x)
    sigma = np.std(x)
    fwhm = 2.355*sigma/mu
    e_event = 41 # keV
    eres = fwhm*e_event
    return fwhm*100 # FWHM  # what units is this????

In [58]:
nfiles = 2
#dirname = "/n/holystore01/LABS/guenette_lab/Users/tcontreras/nexus-production/test/"
files = ["flex.kr83m."+str(i)+".nexus.h5" for i in range(1,2)]
#files_8 = ["/Users/taylorcontreras/Development/Research/Odyssey_sipmstudy/pitch_8mm/flex.kr83m."+str(i)+".nexus.h5" for i in range(0,nfiles)]
config = pd.read_hdf(files[0], "MC/configuration")
tbin_sipms = 1.0 #mus
t1_s1 = 99 #  mus
t2_s1 = 102 #mus
t1_s2 = 102 #mus

test_file = ["/Users/taylorcontreras/Development/Research/Odyssey_sipmstudy/test/noteflon_15mm.h5"]


In [39]:
sns_response = pd.read_hdf(test_file[0], 'MC/sns_response')
sns_positions = pd.read_hdf(test_file[0], 'MC/sns_positions')
sns_positions = sns_positions.drop_duplicates(subset='sensor_id')
sns_positions

,sensor_id,sensor_name,x,y,z
0,29,PmtR11410,250.040558,-216.482941,1468.699951
1,16,PmtR11410,0.040551,-216.529770,1468.699951
2,52,PmtR11410,0.081102,-433.059540,1468.699951
3,40,PmtR11410,-250.000000,-433.012695,1468.699951
4,12,PmtR11410,187.500000,108.300003,1468.699951
...,...,...,...,...,...
15318,2910,TP_SiPM,199.500000,-311.500000,-23.301001
15320,13590,TP_SiPM,-59.500000,262.500000,-23.301001
15321,9253,TP_SiPM,-73.500000,31.500000,-23.301001
15322,6499,TP_SiPM,66.500000,-108.500000,-23.301001


In [ ]:
pd.read_hdf(test_file[0], "MC/configuration")

In [ ]:
#config

In [59]:
all_sipms = pd.DataFrame()
all_pmts = pd.DataFrame()
events_allsensors = pd.DataFrame()
events_allsipms = pd.DataFrame()
events_allpmts = pd.DataFrame()
for file in test_file:
    sns_response = pd.read_hdf(file, 'MC/sns_response')
    sns_positions = pd.read_hdf(file, 'MC/sns_positions')
    print("len of data", sns_response)
    # Sort to get the sipms
    sns_pos_sorted = sns_positions.sort_values(by=['sensor_id'])
    sipm_positions = sns_pos_sorted[sns_pos_sorted["sensor_name"].str.contains("SiPM")]

    # Sort to sum up all charges for each sipms
    sns_response_sorted = sns_response.sort_values(by=['sensor_id'])
    sipm_response = sns_response_sorted.loc[sns_response_sorted["sensor_id"] >999]
    response_byid = sipm_response.groupby('sensor_id')
    summed_charges = response_byid.agg({"charge":"sum"}) 

    # Make data frame with sipms ids, position, and total charge
    print("summed_charges:",summed_charges)
    print("sipms_positions:", sipm_positions)
    new_sipm_positions = sipm_positions.set_index('sensor_id')
    new_df = pd.concat([new_sipm_positions.iloc[:,1:5], summed_charges], axis=1)
    all_sipms = all_sipms.append(new_df)
    
    # Sum of charges in an event
    response_byevent = sns_response_sorted.groupby('event_id')
    event_charge = response_byevent.agg({"charge":"sum"})
    events_allsensors = events_allsensors.append(event_charge)
    
    # Sum of charges in sipms per event
    this = sipm_response.groupby('event_id')
    charges = this.agg({"charge":"sum"})
    events_allsipms = events_allsipms.append(charges)
    
    # pmts
    pmt_response = sns_response_sorted.loc[sns_response_sorted["sensor_id"] < 60]
    response_byid_pmt = pmt_response.groupby('sensor_id')
    summed_charges_pmt = response_byid_pmt.agg({"charge":"sum"})
    pmt_positions = sns_pos_sorted[sns_pos_sorted["sensor_name"].str.contains("Pmt")]
    new_pmt_positions = pmt_positions.set_index('sensor_id')
    new_df_pmt = pd.concat([new_pmt_positions.iloc[:,1:5], summed_charges_pmt], axis=1)
    all_pmts = all_pmts.append(new_df_pmt)
    
    this_pmts = pmt_response.groupby('event_id')
    charges_pmt = this_pmts.agg({"charge":"sum"})
    events_allpmts = events_allpmts.append(charges_pmt)

len of data          event_id  sensor_id  time_bin  charge
0             100         28         0       1
1             100         28         1       1
2             100         28         4       1
3             100         28     32429       1
4             100         28     32476       1
...           ...        ...       ...     ...
1047044       199       2040        90       1
1047045       199       1161        89       1
1047046       199       2461        88       1
1047047       199       1303        91       1
1047048       199       1278        90       1

[1047049 rows x 4 columns]
summed_charges:            charge
sensor_id        
1000          135
1001          138
1002          131
1003          159
1004          143
...           ...
4303          137
4304          148
4305          147
4306          131
4307          121

[3308 rows x 1 columns]
sipms_positions:       sensor_id sensor_name      x      y          z
1748       1000     TP_SiPM   -7.5 -487.5 -23.30100

In [41]:
all_sipms8 = pd.DataFrame()
all_pmts8 = pd.DataFrame()
events_allsensors8 = pd.DataFrame()
events_allsipms8 = pd.DataFrame()
events_allpmts8 = pd.DataFrame()
for file in [test_file]:
    sns_response = pd.read_hdf(file, 'MC/sns_response')
    sns_positions = pd.read_hdf(file, 'MC/sns_positions')

    # Sort to get the sipms
    sns_pos_sorted = sns_positions.sort_values(by=['sensor_id'])
    sipm_positions = sns_pos_sorted[sns_pos_sorted["sensor_name"].str.contains("SiPM")]

    # Sort to sum up all charges for each sipms
    sns_response_sorted = sns_response.sort_values(by=['sensor_id'])
    sipm_response = sns_response_sorted.loc[sns_response_sorted["sensor_id"] >999]
    response_byid = sipm_response.groupby('sensor_id')
    summed_charges = response_byid.agg({"charge":"sum"}) 

    # Make data frame with sipms ids, position, and total charge
    new_sipm_positions = sipm_positions.set_index('sensor_id')
    new_df = pd.concat([new_sipm_positions.iloc[:,1:5], summed_charges], axis=1)
    all_sipms8 = all_sipms8.append(new_df)
    
    # Sum of charges in an event
    response_byevent = sns_response_sorted.groupby('event_id')
    event_charge = response_byevent.agg({"charge":"sum"})
    events_allsensors8 = events_allsensors8.append(event_charge)
    
    # Sum of charges in sipms per event
    this = sipm_response.groupby('event_id')
    charges = this.agg({"charge":"sum"})
    events_allsipms8 = events_allsipms8.append(charges)
    
    # pmts
    pmt_response = sns_response_sorted.loc[sns_response_sorted["sensor_id"] < 60]
    response_byid_pmt = pmt_response.groupby('sensor_id')
    summed_charges_pmt = response_byid_pmt.agg({"charge":"sum"})
    pmt_positions = sns_pos_sorted[sns_pos_sorted["sensor_name"].str.contains("Pmt")]
    new_pmt_positions = pmt_positions.set_index('sensor_id')
    new_df_pmt = pd.concat([new_pmt_positions.iloc[:,1:5], summed_charges_pmt], axis=1)
    all_pmts8 = all_pmts8.append(new_df_pmt)
    
    this_pmts = pmt_response.groupby('event_id')
    charges_pmt = this_pmts.agg({"charge":"sum"})
    events_allpmts8 = events_allpmts8.append(charges_pmt)

NotImplementedError: Support for generic buffers has not been implemented.

In [ ]:
def DrawSiPM(t, s):
    # drawing first side
    t.forward(s) # Forward turtle by s units
    t.left(90) # Turn turtle by 90 degree

    # drawing second side
    t.forward(s) # Forward turtle by s units
    t.left(90) # Turn turtle by 90 degree

    # drawing third side
    t.forward(s) # Forward turtle by s units
    t.left(90) # Turn turtle by 90 degree

    # drawing fourth side
    t.forward(s) # Forward turtle by s units
    t.left(90) # Turn turtle by 90 degree

In [ ]:
import turtle
#x = [x for x in sipm_positions.x]
#y = [y for y in sipm_positions.y]
s = 5 # mm
t = turtle.Turtle() 
window = turtle.Screen()
window.bgcolor("white")
window.exitonclick()

In [ ]:
DrawSiPM(t, s)
t.circle(10)

In [ ]:
import pygame

pygame.init()

white = (255,255,255)
black = (0,0,0)

red = (255,0,0)
green = (0,255,0)
blue = (0,0,255)

gameDisplay = pygame.display.set_mode((800,600))
gameDisplay.fill(black)

In [ ]:
plt.hist(events_allsensors.charge, label="10mm pitch")
#plt.hist(events_allsensors8.charge, label='8mm pitch')
plt.xlabel("Charge per event [pes]")
plt.title("NEXT-100, 6mm sipms")
plt.legend()
plt.show()

plt.hist(events_allsensors.charge, label="All sensors", histtype="step", bins=50, range=(0,16000), linewidth=2)
plt.hist(events_allsipms.charge, label="SiPMs", histtype="step", bins=50, range=(0,16000), linewidth=2)
plt.hist(events_allpmts.charge, label="PMTs", histtype="step", bins=50, range=(0,16000), linewidth=2)
plt.xlabel("Charge per event [pes]")
plt.title("NEXT-100, 6mm sipms, 10mm pitch")
plt.legend()
plt.show()

#plt.hist(events_allsensors8.charge, label="All sensors", histtype="step", bins=50, range=(0,16000), linewidth=2)
#plt.hist(events_allsipms8.charge, label="SiPMs", histtype="step", bins=50, range=(0,16000), linewidth=2)
#plt.hist(events_allpmts8.charge, label="PMTs", histtype="step", bins=50, range=(0,16000), linewidth=2)
#plt.xlabel("Charge per event [pes]")
#plt.title("NEXT-100, 6mm sipms, 8mm pitch")
#plt.legend()
#plt.show()

# Plot sipm positions
#XYrange       =  -500, 500
#hist2d(sipm_positions.x, sipm_positions.y, (25,25), [XYrange, XYrange])
#plt.plot(sipm_positions.x, sipm_positions.y, ".")
#plt.xlim(-50,50)
#plt.ylim(-50,50)
#plt.savefig("sipm_positions.png")
#plt.close()

nbins = 500/10
h = hist2d(all_sipms.x, all_sipms.y, (nbins, nbins), weights = all_sipms.charge)
labels("X [mm]", "Y [mm]", "SiPMs Light Distribution \n (NEXT-100, 6mm sipms, 10mm pitch)")

h2 = hist2d(all_pmts.x, all_pmts.y, (nbins, nbins), weights = all_pmts.charge)
labels("X [mm]", "Y [mm]", "PMTs Light Distribution \n (NEXT-100)")

In [ ]:
print(EnergyRes(events_allsensors.charge),EnergyRes(events_allpmts.charge), EnergyRes(events_allsipms.charge))

In [ ]:
y = events_allsipms8.charge.nunique()

In [ ]:
plt.plot(events_allsipms8.apply(lambda x: (x//100)*100).sort_values('charge').charge.value_counts(), 'o')

In [ ]:
x = events_allsipms8.apply(lambda x: (x//100)*100).sort_values('charge').charge
y = x.value_counts()

In [ ]:
def gaus(x,a,x0,sigma):
    return a*np.exp(-(x-x0)**2/(2*sigma**2))

In [ ]:
#popt, pcov = curve_fit(gaus, x.unique(), y)
mean,std=norm.fit(events_allsipms8)

In [ ]:
mean, std